A particle must move from A to B, using minimal energy, and it must stay on a road.\
It should start, pass through and stop at gates of the from $ $((C|u), (C|o))$

In [1]:
import sympy.physics.mechanics as me
import time
import numpy as np
import sympy as sm
from scipy.interpolate import CubicSpline

from opty.direct_collocation import Problem
from opty.utils import parse_free, create_objective_function
import matplotlib.pyplot as plt
import matplotlib
from IPython.display import HTML
matplotlib.rcParams['animation.embed_limit'] = 2**128
from matplotlib.animation import FuncAnimation

Set up **Kane's equations of motion**

In [ ]:
start = time.time()

N = sm.symbols('N', cls= me.ReferenceFrame)
t = me.dynamicsymbols._t
O, Dmc = sm.symbols('O Dmc', cls= me.Point)
O.set_vel(N, 0)

q0, qf  = me.dynamicsymbols('q_0 q_f')
u0, uf  = me.dynamicsymbols('u_0 u_f')
x, y    = me.dynamicsymbols('x y')
ux, uy  = me.dynamicsymbols('u_x u_y')
Fx, Fy  = me.dynamicsymbols('F_x, F_y')

reibung = sm.symbols('reibung')

m = sm.symbols('m')

Dmc.set_pos(O, x*N.x + y*N.y)
Dmc.set_vel(N, ux*N.x + uy*N.y)

body = me.Particle('body', Dmc, m)
BODY = [body]


# Forces
FL = [(Dmc, Fx*N.x + Fy*N.y -reibung * Dmc.vel(N))]

kd = sm.Matrix([ux - x.diff(t), uy - y.diff(t)])

q_ind = [x, y]
u_ind = [ux, uy]

# Create the KanesMethod object
KM = me.KanesMethod(
                    N,
                    q_ind=q_ind,
                    u_ind=u_ind,
                    kd_eqs=kd,
                    )
(fr, frstar) = KM.kanes_equations(BODY, FL)
eom = fr + frstar
eom = kd.col_join(eom)
eom

Set up so the particle stays on the street\
The street is described by two functions in the X / Y plane f(x), g(x)
with f(x) < g(x) for all x.\
$ f(x) < y < g(x_i)$ is required for the particle to stay on the road.\
As *opty* at present does not allow inequalities in the eoms, I circumvent this by
introducing additional state variables.

In [ ]:
XX, a, b, c, d = sm.symbols('XX a b c d')
def street(XX, a, b, c):
    return a*sm.sin(b*XX) + c

py_upper, py_lower = me.dynamicsymbols('py_upper, py_lower')


delta_d = y - street(x, a, b, c)
delta_u = -y + street(x, a, b, c+d)

eom_add = sm.Matrix([
    -py_upper + delta_d,
    -py_lower + delta_u,
])

eom = eom.col_join(eom_add)
eom

*opty* presently does not support bounds for instance constraints.\
I circumvent this as follows:\
I add two state variables $top$ and $bottom$ and two specified symbolds $top_h$
and $bottom_h$\
Say, we need $a \leq y(t_x) \leq b, a, b > 0 $, then I add this to the equations of motion:

$top(t) = top_h(t) \cdot y(t)$\
$bottom(t) = bottom_h(t) \cdot y(t)$

Then I bound $top_h \in (1.0, \inf)$ and 
$bottom_h \in (0.0, 1.0)$, and put instance constraints $top(t_x) = b, 
bottom(t_x) = a$\
This forces $a \leq y(t_x) \leq b$


In [ ]:
top, bottom = me.dynamicsymbols('top, bottom')
top_h, bottom_h = me.dynamicsymbols('top_h, bottom_h')

eom_add = sm.Matrix([
    -top + y * top_h,
    -bottom + y * bottom_h,
])

eom = eom.col_join(eom_add)
eom

Set up and solve the optimization without the restricting gates

In [ ]:
state_symbols = [x, y, ux, uy, py_upper, py_lower,top, bottom]

laenge = len(state_symbols)
constant_symbols = (m, reibung, a, b, c, d)
specified_symbols = [Fx, Fy, top_h, bottom_h]
unknown_symbols = ()

# Specify the known system parameters.
par_map = {}
par_map[m] = 1.0
par_map[reibung] = 0.5
# below for the shape of the street
par_map[a] = 3.5
par_map[b] = 0.5
par_map[c] = 4.0
par_map[d] = 4.5

t0, tf = 0.0, 10.0
num_nodes = 601
interval_value = (tf - t0)/(num_nodes - 1)

objective = sm.Integral(Fx**2 + Fy**2, t)
obj, obj_grad = create_objective_function(
    objective,
    state_symbols,
    specified_symbols,
    tuple(),
    num_nodes,
    interval_value,
)

instance_constraints = (
        x.func(t0) + 10.0,
        ux.func(t0),
        uy.func(t0),

        x.func(tf) - 10.0,
        ux.func(tf),
        uy.func(tf)-0.0123,
)

grenze = 100.0
grenze1 = 5.0
delta = np.pi/4.0
bounds = {
        Fx: (-grenze, grenze),
        Fy: (-grenze, grenze),
        x: (-14, 14),
        y: (0.0, 25),
        top_h: (1.0, 10.0),
        bottom_h: (0.0, 1.0),
        py_upper: (0.0, 10.0),
        py_lower: (0.0, 10.0),
}

prob = Problem(
        obj,
        obj_grad,
        eom,
        state_symbols,
        num_nodes,
        interval_value,
        known_parameter_map=par_map,
        instance_constraints=instance_constraints,
        bounds=bounds,
)

initial_guess = np.ones(prob.num_free) * 0.01
initial_guess = np.load('point_on_street_no_gate_solution.npy')
prob.add_option('max_iter', 3000)
for i in range(1):
# Find the optimal solution.
    solution_none, info = prob.solve(initial_guess)
    initial_guess = solution_none
    print(f'{i+1} - th iteration')
    print('message from optimizer:', info['status_msg'])
    print('Iterations needed',len(prob.obj_value))
    print(f"objective value {info['obj_val']:.3e} \n")
np.save('point_on_street_no_gate_solution.npy', solution_none)
obj_value_none = prob.objective(solution_none)
prob.plot_objective_value()
prob.plot_constraint_violations(solution_none)

Add the restricting gates to the problem.

In [ ]:
# Add the gates
start_o = 11.0
start_u = 10.0

middle_o = 3.123
middle_u = 2.12345

final_o = 3.1357
final_u = 2.1357

# add additional times
tm = 223 * interval_value
# For t01 = t0 it will not work.
t01 = 1 * interval_value


instance_constraints = instance_constraints + (
        top.func(t01) - start_o,
        bottom.func(t01) - start_u,
        top.func(tm) - middle_o,
        bottom.func(tm) - middle_u,
        top.func(tf) - final_o,
        bottom.func(tf) - final_u,
)

prob = Problem(
        obj,
        obj_grad,
        eom,
        state_symbols,
        num_nodes,
        interval_value,
        known_parameter_map=par_map,
        instance_constraints=instance_constraints,
        bounds=bounds,
)

initial_guess = solution_none
initial_guess = np.load('point_on_street_gate_solution.npy')
prob.add_option('max_iter', 3000)
for i in range(1):
# Find the optimal solution.
    solution, info = prob.solve(initial_guess)
    initial_guess = solution
    print(f'{i+1} - th iteration')
    print('message from optimizer:', info['status_msg'])
    print('Iterations needed',len(prob.obj_value))
    print(f"objective value {info['obj_val']:.3e} \n")
np.save('point_on_street_gate_solution.npy', solution)
prob.plot_objective_value()
print(f'the gates require {(prob.objective(solution) -
      obj_value_none)/obj_value_none *100:.1f} % more energy for the course')

Plot the constraint violations.

In [ ]:
prob.plot_constraint_violations(solution)

Plot generalized coordinates / speeds and forces / torques

In [ ]:
prob.plot_trajectories(solution)

**animation**\
The green arrows symbolize the forces which opty calculated to drive the car. They are perpendicular to the rear axle.

In [ ]:
fps = 15
street_lam = sm.lambdify((x, a, b, c), street(x, a, b, c))

def add_point_to_data(line, x, y):
# to trace the path of the point. Copied from Timo.
    old_x, old_y = line.get_data()
    line.set_data(np.append(old_x, x), np.append(old_y, y))

sol_none, _, _ = parse_free(solution_none, len(state_symbols), len(specified_symbols),
        num_nodes)
sol_none = sol_none.T

state_vals, input_vals, _ = parse_free(solution, len(state_symbols),
                len(specified_symbols), num_nodes)
t_arr = np.linspace(t0, tf, num_nodes)
state_sol = CubicSpline(t_arr, state_vals.T)
input_sol = CubicSpline(t_arr, input_vals.T)

# end point of the force
Fbq = me.Point('Fbq')
fx, fy = sm.symbols('fx, fy')

Fbq.set_pos(Dmc, fx*N.x + fy*N.y)

coordinates = Dmc.pos_from(O).to_matrix(N)
coordinates = coordinates.row_join(Fbq.pos_from(O).to_matrix(N))

pL, pL_vals = zip(*par_map.items())
coords_lam = sm.lambdify((*state_symbols, fx, fy, *pL), coordinates,
        cse=True)

def init():
        xmin, xmax = -15, 15.
        ymin, ymax = -0.75, 12.5

        fig = plt.figure(figsize=(8, 8))
        ax  = fig.add_subplot(111)
        ax.set_xlim(xmin, xmax)
        ax.set_ylim(ymin, ymax)
        ax.set_aspect('equal')
        ax.grid()

        XX = np.linspace(xmin, xmax,  200)
        ax.fill_between(XX, ymin, street_lam(XX, par_map[a], par_map[b],
                par_map[c]), color='grey', alpha=0.25)
        ax.fill_between(XX, street_lam(XX, par_map[a], par_map[b],
                par_map[c]+par_map[d]), ymax, color='grey', alpha=0.25)

        ax.plot(XX, street_lam(XX, par_map[a], par_map[b], par_map[c]),
                color='black')
        ax.plot(XX, street_lam(XX, par_map[a],
                par_map[b], par_map[c]+par_map[d]), color='black')
        ax.vlines(-10.0, street_lam(-10.0, par_map[a], par_map[b],
                par_map[c]), street_lam(-10, par_map[a], par_map[b],
                par_map[c]+par_map[d]), color='red', linestyle='dotted')

        coords = coords_lam(*state_sol(t01), input_sol(t01)[0], input_sol(t01)[1], *pL_vals)
        ax.vlines(coords[0, 0], street_lam(coords[0, 0], par_map[a], par_map[b], par_map[c]),
                start_u, color='red', linestyle='-')
        ax.vlines(coords[0, 0], start_o,
                street_lam(coords[0, 0], par_map[a], par_map[b], par_map[c]+par_map[d]
                ), color='red', linestyle='-')
        ax.vlines(coords[0, 0], start_u, start_o, color='red', linestyle='dotted')

        coords = coords_lam(*state_sol(tm), input_sol(tm)[0], input_sol(tm)[1], *pL_vals)
        ax.vlines(coords[0, 0], street_lam(coords[0, 0], par_map[a], par_map[b], par_map[c]),
                middle_u, color='blue', linestyle='-')
        ax.vlines(coords[0, 0], middle_o,
                street_lam(coords[0, 0], par_map[a], par_map[b], par_map[c]+par_map[d]
                ), color='blue', linestyle='-')
        ax.vlines(coords[0, 0], middle_u, middle_o, color='blue', linestyle='dotted')

        coords = coords_lam(*state_sol(tf), input_sol(tf)[0], input_sol(tf)[1], *pL_vals)
        ax.vlines(coords[0, 0], street_lam(coords[0, 0], par_map[a], par_map[b], par_map[c]),
                final_u, color='green', linestyle='-')
        ax.vlines(coords[0, 0], final_o,
                street_lam(coords[0, 0], par_map[a], par_map[b], par_map[c]+par_map[d]
                ), color='green', linestyle='-')
        ax.vlines(coords[0, 0], final_u, final_o, color='green', linestyle='dotted')

        ax.plot(sol_none[:, 0], sol_none[:, 1], color='black', linestyle='dotted')


        line1, = ax.plot([], [], color='red', marker='o', markersize=5)
        line4  = ax.quiver([], [], [], [], color='green', scale=10, width=0.004,
                headwidth=8)

        return fig, ax, line1, line4

# Function to update the plot for each animation frame
fig, ax, line1, line4 = init()
def update(t):
    message = (f'running time {t:.2f} sec \n The driving/breaking force is green \n' +
               f'The dotted black is the optimum path without gates')
    ax.set_title(message, fontsize=12)

    coords = coords_lam(*state_sol(t), input_sol(t)[0], input_sol(t)[1], *pL_vals)

    #   Pb, Dmc, Pf, Pbl, Pbr, Pfl, Pfr, Fbq
    line1.set_data([coords[0, 0]], [coords[1, 0]])

    line4.set_offsets([coords[0, 0], coords[1, 0]])
    line4.set_UVC(coords[0, 1] - coords[0, 0] , coords[1, 1] - coords[1, 0])
    return line1, line4,

frames = np.linspace(t0, tf, int(fps * (tf - t0)))
animation = FuncAnimation(fig, update, frames=frames, interval=1000 / fps)


# Show the animation
display(HTML(animation.to_jshtml()))
plt.close()
